In [6]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import requests
import time
from shapely.geometry import box

# --- 1. Baixar o polígono do arthur alvim ---
bairro = ox.geocode_to_gdf("Arthur Alvim, São Paulo")

# --- 2. Criar a grade (grid) sobre o bairro ---
def criar_grid(bairro, tamanho=0.005):  # Aproximadamente 200m
    minx, miny, maxx, maxy = bairro.total_bounds
    grid = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            grid.append(box(x, y, x + tamanho, y + tamanho))
            y += tamanho
        x += tamanho
    grid_gdf = gpd.GeoDataFrame(geometry=grid, crs=bairro.crs)
    grid_gdf = gpd.overlay(grid_gdf, bairro, how='intersection')
    return grid_gdf

grid = criar_grid(bairro)


# --- 3. Corrigir projeção para metros, calcular centroide ---
grid = grid.to_crs(epsg=3857)  # Web Mercator
grid["centroide"] = grid.geometry.centroid

# --- 4. Criar colunas separadas de latitude/longitude
centroides = grid["centroide"].to_crs(epsg=4326)  # voltar para lat/lon
grid["centroide_lon"] = centroides.geometry.x
grid["centroide_lat"] = centroides.geometry.y

# --- 6. Definir hospital e API ---
API_KEY = "5b3ce3597851110001cf624824d680a2a89c4dc48933794aaca7b81d"  # Troque aqui pela sua chave
hospital_coords = [-46.4519756, -23.5351716] 

def calcular_distancia_km(origem, destino, api_key):
    url = "https://api.openrouteservice.org/v2/directions/driving-car/json"
    headers = {
        "Authorization": api_key,
        "Content-Type": "application/json"
    }
    body = {
        "coordinates": [origem, destino]
    }
    try:
        response = requests.post(url, json=body, headers=headers)
        if response.status_code == 200:
            dados = response.json()
            distancia_metros = dados['routes'][0]['summary']['distance']
            distancia_km = distancia_metros / 1000
            return round(distancia_km, 2)
        else:
            print("Erro na API:", response.text)
            return None
    except Exception as e:
        print(f"Erro: {e}")
        return None

# --- 7. Calcular a distância para cada grid ---
distancias_km = []
for i, row in grid.iterrows():
    origem = [row["centroide_lon"], row["centroide_lat"]]
    distancia = calcular_distancia_km(origem, hospital_coords, API_KEY)
    distancias_km.append(distancia)
    time.sleep(1)  # evita limite da API gratuita

grid["distancia_km"] = distancias_km
grid.to_csv(f"estimativa_arthur_alvim.csv", index=False)

In [7]:
grid

,bbox_west,bbox_south,bbox_east,bbox_north,place_id,osm_type,osm_id,lat,lon,class,...,place_rank,importance,addresstype,name,display_name,geometry,centroide,centroide_lon,centroide_lat,distancia_km
0,-46.482253,-23.551794,-46.481502,-23.551065,8364972,way,743737297,-23.551432,-46.481877,amenity,...,30,0.000079,amenity,ETEC Tereza Aparecida Cardoso Nunes de Oliveira,ETEC Tereza Aparecida Cardoso Nunes de Oliveir...,"POLYGON ((-5174343.276 -2698872.02, -5174380.7...",POINT (-5174338.982 -2698842.435),-46.481878,-23.55143,4.25


In [3]:
%pip install folium
import folium
from shapely.geometry import mapping

# Converter o bairro para EPSG:4326 (lat/lon) para exibição no mapa
bairro_wgs84 = bairro.to_crs(epsg=4326)
grid_wgs84 = grid.to_crs(epsg=4326)

# Criar mapa centralizado no bairro
centroide_bairro = bairro_wgs84.geometry.centroid.iloc[0]
m = folium.Map(location=[centroide_bairro.y, centroide_bairro.x], zoom_start=14)

# Adicionar polígono do bairro
folium.GeoJson(bairro_wgs84.geometry.__geo_interface__,
               name="Bairro - Arthur alvim",
               style_function=lambda x: {"color": "black", "fillOpacity": 0.1}
).add_to(m)

# Adicionar os grids
for _, row in grid_wgs84.iterrows():
    folium.GeoJson(mapping(row.geometry),
                   style_function=lambda x: {"color": "blue", "weight": 1, "fillOpacity": 0}
    ).add_to(m)

# Adicionar ponto do hospital
folium.Marker(
    location=[-23.5351716, -46.4519756] ,
    popup="Hospital Itaquera",
    icon=folium.Icon(color='red', icon='plus-sign')
).add_to(m)

# Mostrar mapa
m


[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


C:\Users\joth1\AppData\Local\Temp\ipykernel_16108\355286289.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroide_bairro = bairro_wgs84.geometry.centroid.iloc[0]
